# Usando Regressão Logística, Árvores de Decisão e Boosting para predição da eleição de deputados

* Imports gerais
* Modelos
* Métricas de avaliação

In [1]:
%%capture

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import normaltest


try:
  from google.colab import files, drive
  drive.mount('/content/drive')
except:
  pass

In [2]:
%%capture
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.svm import LinearSVC as LSVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

In [3]:
%%capture
from sklearn.metrics import accuracy_score, auc, confusion_matrix
from sklearn.metrics import precision_score, recall_score

## Import dos dados

Tentamos importar os dados a partir do Google Drive, já que está sendo feito no Google Colaboratory. Caso não seja possível, importa-se os dados a partir da pasta local.

In [4]:
try:
  drv = 'drive/My Drive/CDP/'
  treino = pd.read_csv(drv + 'train_cat.csv')
  teste = pd.read_csv(drv + 'test_cat.csv')
except:
  treino = pd.read_csv('train_cat.csv')
  teste = pd.read_csv('test_cat.csv')

In [5]:
treino.columns.to_series().groupby(treino.dtypes).groups

{dtype('int64'): Index([u'ano', u'sequencial_candidato', u'quantidade_doacoes',
        u'quantidade_doadores', u'quantidade_despesas',
        u'quantidade_fornecedores'],
       dtype='object'),
 dtype('float64'): Index([u'total_receita', u'media_receita',
        u'recursos_de_outros_candidatos.comites',
        u'recursos_de_pessoas_fisicas', u'recursos_de_pessoas_juridicas',
        u'recursos_proprios', u'recursos_de_partido_politico', u'total_despesa',
        u'media_despesa'],
       dtype='object'),
 dtype('O'): Index([u'nome', u'uf', u'partido', u'cargo', u'sexo', u'grau', u'estado_civil',
        u'ocupacao', u'situacao'],
       dtype='object')}

## Análise dos dados
Como analisaremos os dados de treino, é necessário criar funções que sejam reutilizáveis, para permitir que o mesmo procedimento seja realizado em novas etapas do código. Inicialmente, analisaremos a distribuição das variáveis numéricas com a função `analisa_numericas`, aplicando a normalização nos casos necessários, vistos posteriormente.

Além dos gráficos, será feita uma verificação matemática para gerar um array no qual serão definidas as variáveis que devem ser normalizadas nos dados de treino e de teste: 
considera-se a hipótese de que o array segue uma distribuição normal e define-se um limite `alpha=0.0001` ; caso o resultado do teste de normalidade for menor que alpha, a hipótese de seguir a distribuição normal é rejeitada, isto é, os dados não seguem uma distribuição normal.

In [ ]:
def analisa_numericas(df):
  plt.figure(figsize=(16,24))
  plt.title('Análise das variáveis')
  col_numericas = []
  col_nao_normais = []
  alpha = 1e-3
  
  for coluna in list(df):
    if (isinstance(df[coluna][0], np.int64) or isinstance(df[coluna][0], np.float64)) \
      and coluna != "ano" and coluna != "sequencial_candidato":
      col_numericas.append(coluna)
      if normaltest(df[coluna])[1] < alpha:
        col_nao_normais.append(coluna)
        
  for i in range(len(col_numericas)):
    coluna = col_numericas[i]
    plt.subplot(round(len(col_numericas)/2.0)+1, 2, i+1)
    plt.title(coluna)
    maxx = int(max(df[coluna]))
    plt.hist(df[coluna], maxx)
  plt.show()
  
  return col_nao_normais
  
colunas = analisa_numericas(treino)

## Manipulação dos Dados

A partir da análise dos gráficos acima, podemos concluir que nenhuma coluna numérica possui seus dados normalizados, e por isso criaremos a função `normaliza_dados`,  que normaliza todas as colunas recebidas e, ao fim, repete a análise gráfica feita acima.

In [0]:
def normaliza_dados(df, colunas):
  for col in colunas:
    df[col] = np.log(df[col])
  analisa_numericas(df)
  return df
  
    
treino = normaliza_dados(treino, colunas)

Agora que todos os dados numéricos passaram por uma normalização, podemos iniciar a análise dos dados não numéricos. Assim como no lab anterior, iremos descartar a coluna com os nomes dos deputados por não fazer sentido que o nome beneficie ou prejudique a eleição de um deputado.

In [0]:
try:
  treino = treino.drop('ocupacao', axis=1)
  teste = teste.drop('ocupacao', axis=1)
  treino = treino.drop('nome', axis=1)
  teste = teste.drop('nome', axis=1)
except:
  pass

Em seguida, transformamos todas as colunas de fatores em variáveis dummy

In [0]:
def transforma_fatores(df):
  fatores = ['ano', 'uf', 'partido', 'cargo', 'sexo', 'estado_civil']
  for fator in fatores:
    df = pd.concat([df, pd.get_dummies(df[fator])], axis=1)
    df = df.drop(fator, axis=1)
  return df

treino = transforma_fatores(treino)

Como a escolaridade é um fator que, quanto maior, mais impacto, a codificamos de uma forma diferente, seguindo a função abaixo.

In [0]:
def codifica_escolaridade(dataframe):
  niv_escolaridade = {'LÊ E ESCREVE': 0, 'ENSINO FUNDAMENTAL INCOMPLETO': 1,
                      'ENSINO FUNDAMENTAL COMPLETO': 2,
                      'ENSINO MÉDIO INCOMPLETO': 3, 'ENSINO MÉDIO COMPLETO': 4,
                      'SUPERIOR INCOMPLETO': 5, 'SUPERIOR COMPLETO': 6}

  ordenar_grau = []

  for i, row in dataframe.iterrows():
      ordenar_grau.append(niv_escolaridade[row['grau']])

  dataframe['grau'] = ordenar_grau
  return dataframe

treino = codifica_escolaridade(treino)

## Treinando os modelos

In [0]:
treino.head()

In [0]:
tree = DTC()
x, y = treino.drop('situacao', axis=1), treino['situacao']
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size=0.3)

tree.fit(x_treino, y_treino)
pred = tree.predict(x_test)

In [0]:
acc = accuracy_score(y_teste, pred, normalize = False)
prec = precision_score(y_teste, pred, normalize = False)
recall = recall_score(y_teste, pred, normalize = False)
print(acc, prec, recall)